In [1]:
%matplotlib

Using matplotlib backend: Qt4Agg


# Dithering en image avec Python

Dans ce projet, nous nous basons sur des méthodes utilisées pour le dithering en traitement d'image [comme dans cette méthode](http://michal.is/projects/image-dithering-in-matlab/). Ces méthodes fonctionnant sous Matlab, nous avons donc entrepris leur analyse et leur réécriture en méthodes Python

## Outils

Pour le traitement d'image, nous utilisons principalement des outils SciPy comme NumPy ou Matplotlib, qui est utilisé pour l'affichage des images. Nous utilisons également skimage qui permet d'accéder à des méthodes notamment de conversion de couleur (grayscale) et du thresholding.

## Seuillage

Dans cette première partie, on expose deux méthodes basiques qui sont le seuillage fixé

In [3]:
from skimage import io, color, filters
import matplotlib.pyplot as plt
import threshold


img = io.imread("lenna.png")
img_gray = color.rgb2gray(img)

bin_image_fixed = threshold.fixedThresholding(img_gray, 120/255)
plt.figure(1)
plt.imshow(bin_image_fixed, cmap='Greys_r')


bin_image_random = threshold.randomThresholding(img_gray, 0.3)
plt.figure(2)
plt.imshow(bin_image_random, cmap='Greys_r')